In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=7d22773e8e204c92b2ec44b2ee07d0e404cfa61186bce6bb8167f691d1216109
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F24 '''
def F24(X):
    f = bn.F24()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.02 ,random_state=0 , l1_ratio=0.98, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_242000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_24_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=889891, Tue Apr 14 13:35:33 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 4.558990443450727e+03 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 4.538000367170872e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 4.533720576075478e+03 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4601.73794474] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4565.23280597] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   75   1275 3.178003449983618e+03 1.2e+00 8.30e-01  8e-01  8e-01 0:03.1
  100   1700 2.839112615822321e+03 1.2e+00 5.19e-01  5e-01  5e-01 0:04.1
  200   3400 2.469017878682298e+03 1.2e+00 8.39e-02  8e-02  8e-02 0:08.2
  300   5100 2.460102146039535e+03 1.3e+00 1.20e-02  1e-02  1e-02 0:12.3
  400   6800 2.459938501227151e+03 1.3e+00 1.77e-03  2e-03  2e-03 0:16.4
  500   8500 2.459934820454748e+03 1.3e+00 3.07e-04  3e-04  3e-04 0:20.4
  600  10200 2.459934690205300e+03 1.3e+00 5.06e-05  5e-05  5e-05 0:24.5
  700  11900 2.459934686532722e+03 1.3e+00 8.11e-06  7e-06  8e-06 0:28.5
  800  13600 2.459934686450311e+03 1.3e+00 1.41e-06  1e-06  1e-06 0:32.5
  900  15300 2.459934686448059e+03 1.3e+00 2.14e-07  2e-07  2e-07 0:36.6
 1000  17000 2.459934686447994e+03 1.3e+00 4.00e-08  4e-08  4e-08 0:40.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 2.459934686447992e+03 1.4e+00 1.60e-08  1e-08  2e-08 0:44.7
 1142  19414 2.459934686447992e+03 1.4e+00 1.27e-08 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4647.2838846] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4647.283885, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4647.283885, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')


    2     34 4.348873671324096e+03 1.0e+00 2.32e+00  2e+00  2e+00 0:00.1
    3     51 4.347652687156327e+03 1.0e+00 2.26e+00  2e+00  2e+00 0:00.1


...

   75   1275 4.031537743833102e+03 1.2e+00 1.28e+00  1e+00  1e+00 0:03.1
  100   1700 3.303062884564895e+03 1.2e+00 8.43e-01  8e-01  9e-01 0:04.2
  200   3400 2.486373637803627e+03 1.3e+00 1.33e-01  1e-01  1e-01 0:08.3
  300   5100 2.460832752224251e+03 1.3e+00 2.45e-02  2e-02  2e-02 0:12.4
  400   6800 2.459959212481648e+03 1.3e+00 4.18e-03  4e-03  4e-03 0:16.5
  500   8500 2.459935379372559e+03 1.3e+00 7.04e-04  7e-04  7e-04 0:20.6
  600  10200 2.459934715918545e+03 1.3e+00 1.29e-04  1e-04  1e-04 0:24.8
  700  11900 2.459934687018701e+03 1.3e+00 2.01e-05  2e-05  2e-05 0:28.9
  800  13600 2.459934686463992e+03 1.4e+00 3.24e-06  3e-06  3e-06 0:33.0
  900  15300 2.459934686448411e+03 1.4e+00 5.87e-07  5e-07  6e-07 0:37.1
 1000  17000 2.459934686448004e+03 1.4e+00 9.14e-08  8e-08  9e-08 0:41.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 2.459934686447993e+03 1.4e+00 2.31e-08  2e-08  2e-08 0:45.3
 1188  20196 2.459934686447993e+03 1.4e+00 1.35e-08 

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4647.2838846] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4647.283885, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=4647.283885, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')


    2     34 4.155531407485452e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 4.188152818199145e+03 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [4792.76146123] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   63   1071 3.828409683434879e+02 1.3e+00 1.46e+00  1e+00  1e+00 0:03.2
  100   1700 -6.567987679763842e+02 1.3e+00 7.86e-01  8e-01  8e-01 0:04.9
  200   3400 -1.458992137668410e+03 1.3e+00 2.10e-01  2e-01  2e-01 0:09.8
  300   5100 -1.567116054467700e+03 1.4e+00 5.94e-02  6e-02  6e-02 0:14.7
  400   6800 -1.572800592612384e+03 1.5e+00 1.72e-02  2e-02  2e-02 0:19.6
  500   8500 -1.573580983115307e+03 1.5e+00 5.98e-03  5e-03  6e-03 0:24.5
  600  10200 -1.573646036019934e+03 1.6e+00 1.89e-03  2e-03  2e-03 0:29.3
  700  11900 -1.573657702021016e+03 1.8e+00 9.28e-04  8e-04  1e-03 0:34.2
  800  13600 -1.573665877972458e+03 3.0e+00 1.13e-03  9e-04  2e-03 0:39.1
  900  15300 -1.573669592844039e+03 3.6e+00 4.77e-04  4e-04  1e-03 0:44.0
 1000  17000 -1.573669999053029e+03 3.7e+00 1.86e-04  1e-04  5e-04 0:48.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.573670074366300e+03 3.8e+00 6.08e-05  5e-05  2e-04 0:53.7
 1200  20400 -1.573670086547945e+03 3.9e+

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F24_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F24(X_Values.iloc[i,:100])
    SVM_Fun [i] = F24(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F24(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([1376.65827485]), count=array([1])) 0.00017481171796807597
SVM
ModeResult(mode=array([4490.86296715]), count=array([1])) 341.56584402835136
ELN
ModeResult(mode=array([5913.67967059]), count=array([1])) 116.36035801310386
